In [1]:
# Para usar o sleep:
import time
# Para logs:
from tqdm import tqdm
import logging
import sys
sys.path.append('../utils')
from utils import get_logger, Clock
# Para web scraping:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import cchardet
# Para acessar as credenciais de login de um arquivo externo:
import os
import dotenv
# Para navegar na página além do botão 'Show more':
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

Inicializando o logger:

In [2]:
logger = logging.getLogger(__name__)
logger = get_logger(logger=logger)

### Obtendo os links das cervejas brasileiras do _Untappd_:

Fazendo uma requisição GET à página web que contém as cervejas brasileiras na seção _top rated_ da rede social _Untappd_ e transformando o resultado em um objeto BeautifulSoup:

In [4]:
headers = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 OPR/97.0.0.0"
}

url = 'https://untappd.com/beer/top_rated?country=brazil'

page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Top Rated Beers from Brazil – Top Rated Beers - Untappd
  </title>
  <meta content="Untappd" property="og:site_name"/>
  <meta content="Top rated beers from Brazil on Untappd based on user submitted ratings. Filter by country or style to discover new beers." name="description"/>
  <meta content="user-scalable=no, initial-scale=1, maximum-scale=1, minimum-scale=1, width=device-width" name="viewport"/>
  <meta content="origin-when-crossorigin" name="referrer"/>
  <meta content="The Untappd Team" name="author">
   <meta content="Copyright Untappd 2023. All Rights Reserved." name="Copyright"/>
   <meta content="Untappd" name="DC.title"/>
   <meta content="Top rated beers from Brazil on Untappd based on user submitted ratings. Filter by country or style to discover new beers." name="DC.subject"/>
   <meta content="The Untappd Team" name="DC.creator"/>
   <meta content

Minerando os links de todas as cervejas brasileiras e armazenando em uma lista:

In [5]:
brazilian_beer_links = [a_tag.get('href') for a_tag in soup.select('div.beer-item > a:first-child')]
brazilian_beer_links

['/b/spartacus-brewing-iii-three/4283069',
 '/b/spartacus-brewing-iv-four/4756418',
 '/b/spartacus-brewing-v-five/5285289',
 '/b/hocus-pocus-overdrive/1682105',
 '/b/spartacus-brewing-arete-tomo-2/4542383',
 '/b/salvador-brewing-co-cookie-de-chocolate/2908960',
 '/b/hocus-pocus-elephants-graveyard/2769185',
 '/b/cervejaria-dogma-rizoma/1536694',
 '/b/spartacus-brewing-arete-tomo-1/4098457',
 '/b/hocus-pocus-supersymmetry/1930404',
 '/b/spartacus-brewing-etera-tomo-5/4769967',
 '/b/koala-san-brew-cosmic-blur/3452440',
 '/b/trilha-cervejaria-pao-de-mel/3252352',
 '/b/hocus-pocus-rabbit-hole/2321900',
 '/b/spartacus-brewing-fwiw/3905760',
 '/b/cervejaria-dogma-mosaic-lover/1698497',
 '/b/salvador-brewing-co-engesa-oil-coffee-edition/3036359',
 '/b/spartacus-brewing-barrel-aged-icon-double-chocolate-peanut-butter/5076259',
 '/b/5-elementos-cervejaria-artesanal-coffee-and-pancake-brunch-stout/2546467',
 '/b/mad-dwarf-brewery-md-oloroso-2017/2957221',
 '/b/cervejaria-dogma-morning-gringo/265

In [6]:
logger.info(f'Número total de cervejas no top rated do Brasil: {len(brazilian_beer_links)}')

Número total de cervejas no top rated do Brasil: 50
